# Functions and classes


## Modules, packages, libraries

Definitions:

  * Modules:
  A module is a file which contains python functions, global variables etc. It is nothing but a `.py` file which has Python executable code.

  * Packages:
  A package is a collection of Python modules: while a module is a single Python file, a package is a directory of Python modules containing an additional `__init__.py` file, to distinguish a package from a directory that just happens to contain a bunch of Python scripts. Packages can be nested to any depth, provided that the corresponding directories contain their own `__init__.py` file.
  
  * Libraries:
  A library is a collection of various packages. There is no difference between a package and Python library conceptually.
  
Modules/packages/libraries can be easily "imported" and made functional in your python code. A set of basic libriaries comes with every Python installation. Others can be installed on the local machine and then imported. Your own code sitting somewhere else in your local computer can be imported too.

Further details on packages and how to create them can be found on the online Python documentation: for instance for the [math library](https://docs.python.org/3/library/math.html).

In [ ]:
# Import all the content of the math library
import math
print(math.pi)

# You can import a library and label it for convenience
import math as m
print(m.pi)

# Alternatively, you can import only a given object or function from the library instead of the full library
from math import pi, sqrt  # you can add several libraries at once, just list them separated by a ', '
print(pi)

# Importing the full library (try to avoid this if you only need specific functions)
from math import *
print(sqrt(7))

To know which modules are installed and ready to be imported:

In [ ]:
import sys
for k, v in sys.modules.items():
    print(k)

# Alternative for older python versions
# print(help('modules'))

`pip` is a special package. It is used from the command line to install properly (e.g. matching the version of the local packages) new packages:

`pip install matplotlib`

`python3 -m pip install matplotlib`

It can also be used from within Python to check i.e. the set installed packages and their versions.

N.B.: only the installed packages on top of the default ones will be listed.

In [ ]:
import sys

if sys.version_info >= (3, 8):
    from importlib import metadata as importlib_metadata
else:
    import importlib_metadata

dists = importlib_metadata.distributions()
for dist in dists:
    name = dist.metadata["Name"]
    version = dist.version
    print("found distribution %s version %s" % (name, version))

#import pip
#sorted(["%s==%s" % (i.key, i.version) for i in pip.get_installed_distributions()])

## Functions

Defining function in Python is very simple, because you do not have to specify the type of the arguments and of the returned objects, like in statically typed languages. This is both useful and dangerous. Use the `try`/`except` construction to make it safe.

In [ ]:
# default arguments can also be declared
def my_function(a, b = 2):
    r = a + 2 * b
    return r

my_function(3)

Normally, variables created inside the function (*local* variables) do not exist anymore outside the scope of the function itself.

In [ ]:
# print(r) # r is not recognized outside the function

Functions can edit *global* variables as well, i.e. variables that are declared outside the function scope, but within the same module.

The statement ```global``` is used to access the value of the variable outside the function with the same name (not recommended!):

In [ ]:
x = "awesome"

def myfunc():
    x = "fantastic"
    print("Python is " + x)

myfunc()

print("Python is " + x)

In [ ]:
x = "awesome"

def myfunc():
    global x # try to avoid global as much as possible
    x = "fantastic"
    print("Python is " + x)

myfunc()

print("Python is " + x)

Hint: if you have nested functions and you want to access to variables in the parent function but not in the global scope, use `nonlocal`.

You can treat functions just like other Python object, and add them to the data structures like lists, dict, etc.:

In [ ]:
def square(x):
    """Square of x."""
    return x * x

def cube(x):
    """Cube of x."""
    return x * x * x

# create a dictionary of functions
funcs = {
    'square' : square,
    'cube' : cube,
}

x = 3
print(square(x))
print(cube(x))

for func in sorted(funcs):
    print(func, funcs[func](x))

### Function arguments

In other languages, you can choose whether you want want to pass variables to a function by *value* or by *reference*.

Python does not pass arguments neither by *value* nor by *reference*, but by *assignment*. When a function is called, each of the parameters of the function is assigned to the object they were passed in.

The difference in the behaviour of mutable and immutable objects emerges when these objects are passed as an argument to a function.

When an **immutable object** is passed to a function as an argument, and the object is modified inside the function, Python creates a copy of the original object, which is valid only within the scope of the function.

In [ ]:
def modify(x):
    print("Inside the fuction:         ", x, id(x))
    x += 2
    print("After assignment in fuction:", x, id(x))
    return x

x = 1
print("Before calling the function:", x, id(x))
y = modify(x) # call the function
print("Returned by the function:   ", y, id(y))
print("After calling the function: ", x, id(x))

Now let's consider a **mutable object**, e.g. a list `x = [1, 2, 3]`. If within the function the content of `x` is directly changed (e.g. `x[0] = 999`), then `x` changes outside the function as well. 

In [ ]:
def modify(x):
    print("Inside the fuction:         ", x, id(x))
    x[0] = 999
    print("After assignment in fuction:", x, id(x))
    return x

x = [1, 2, 3]
print("Before calling the function:", x, id(x))
y = modify(x) # call the function
print("Returned by the function:   ", y, id(y))
print("After calling the function: ", x, id(x))

However, if `x` is reassigned within the function to a new object (e.g. another list), then the copy of the name `x` now points to the new object, but `x` outside the function is unchanged.

In [ ]:
def no_modify(x):
    print("Inside the fuction:         ", x, id(x))
    x = [4, 5, 6]
    print("After assignment in fuction:", x, id(x))
    return x

x = [1, 2, 3]
print("Before calling the function:", x, id(x))
y = no_modify(x) # call the function
print("Returned by the function:   ", y, id(y))
print("After calling the function: ", x, id(x))

The difference is that in the first case only one element of the object has been reassigned, while in the second the object itself is modified.

#### Initialization of function default arguments

A Python behaviour that may not be intuitive, and you should pay attention to:

In [ ]:
def f(x = []):
    x.append(1)
    return x

print(f())
print(f())
print(f(x = [9, 9, 9]))
print(f())
print(f())

What actually happens is that a new list is created once when the function is defined, and the same list is used in each successive call.

**Python’s default arguments are evaluated once when the function is defined**, not each time the function is called (like it is in say, Ruby). This means that if you use a mutable default argument and mutate it, you will have mutated that object for all future calls to the function as well. Check this [post](https://docs.python-guide.org/writing/gotchas/).

The correct approach is to write the function in such a way that it *creates a new object* each time the function is called, by using a default argument that explicitly states that no argument was provided (`None` is often a good choice).

In [ ]:
def f(x = None):
    if x is None:
        x = []
    x.append(1)
    return x

print(f())
print(f())
print(f(x = [9, 9, 9]))
print(f())
print(f())

#### Shallow copy and Deep copy

Generally speaking, the user can "force" Python to create a completely different object that has no ties with the original object, which is especially useful in case of mutable objects.

Python provides the `copy` module ([link to the documentation](https://docs.python.org/3/library/copy.html)) that allows you to perform two types of copy, the **shallow** and the **deep** copy.

The difference between shallow and deep copying is only relevant for compound objects (objects that contain other objects, like lists or class instances).

- A **shallow copy** constructs a new compound object and then (to the extent possible) inserts references into it to the objects found in the original.

- A **deep copy** constructs a new compound object and then, recursively, inserts copies into it of the objects found in the original.

In [ ]:
import copy

x = [1, 2, 3]

x_copy = copy.copy(x) # shallow copy of x
#x_copy = copy.deepcopy(x) # deep copy of x

x_copy[0] = 999 # modify copy, and check that the original object is unchanged

print("Shallow copy:", x_copy)
print("Original:    ", x)

### Higher order functions

A function that uses another function as an input argument or returns a function is known as a higher-order function (HOF). The most familiar examples are `map` and `filter`.

#### map

The map function applies a function to each member of a collection

In [ ]:
x = list( map(square, range(5)) ) 
print(x)

# Note the difference w.r.t Python 2. In Python 3 map retuns an iterator so you can loop on it like:
for i in map(square, range(5)):
    print(i)

#### filter

The filter function applies a predicate to each member of a collection, retaining only those where the predicate is `True`

In [ ]:
def is_even(x):
    return x % 2 == 0

print(list(filter(is_even, range(5))))

HOF can be nested:

In [ ]:
list(map(square, filter(is_even, range(5))))


#### reduce

The reduce function reduces a collection using a binary operator to combine items two at a time. More often than not, reduce can be substituted with a more efficient `for` loop. It is worth mentioning for itspast key role in big-data applications together with `map` (the map-reduce paradigm, quite common in Hadoop).

N.B.: it no longer exist as built-in function in Python3, it is now part of the `functools` library.

In [ ]:
from functools import reduce

def my_add(x, y):
    print("Adding", x, "and", y)
    return x + y

# another implementation of the sum function
reduce(my_add, [1, 2, 3, 4, 5])

#### zip

zip is useful when you need to iterate over matched elements of multiple lists:

In [ ]:
xs = [1, 2, 3, 4]
ys = [10, 20, 30, 40]
zs = ['a', 'b', 'c', 'd', 'e']

# zip stops at the end of the shortest list
for x, y, z in zip(xs, ys, zs):
    print(x, y, z)

#### Custom HOF

Python allows you to define custom HOF, or in general functions that accept other functions as arguments:

In [ ]:
def custom_sum(xs, transform):
    """Returns the sum of xs after a user specified transform."""
    return sum(map(transform, xs))

xs = range(5)
print(custom_sum(xs, square))
print(custom_sum(xs, cube))

### Recursive functions

A recursive function is a function that calls itself. For instance:

In [ ]:
def factorial(x):
    if x == 1:
        return 1
    else:
        # recursive call to the function
        return x * factorial(x - 1)

factorial(5)

### Anonimous (lambda) functions

When using functional style, there is often the need to create specific functions that perform a limited task as input to a HOF such as map or filter. In such cases, these functions are often written as anonymous or lambda functions. 
The syntax is as follows:

`lambda <arguments> : <expression>`

Hint: do not overcomplicate lambda functions. If you find it hard to understand what a lambda function is doing, it should probably be rewritten as a regular function.

In [ ]:
# Explicit definition
addit = lambda x, y: x + y
addit(3, 4)

In [ ]:
# The function is declared and directly passed to the HOF
for i in map(lambda x: x * x, range(5)): print(i)

In [ ]:
# mid-lecture test: could you tell what this function does?
from functools import reduce
s1 = reduce(lambda x, y: x + y, map(lambda x: x**2, range(1, 10)))
#print(s1)

### Decorators

Decorators are a type of HOF that *takes a function(s) as argument* and return *the wrapped function* that provides additional useful properties.

Examples:

  - Logging
  - Just-In-Time (JIT) compilation
  - GPU acceleration ([numba](https://numba.pydata.org/))
  - ...
  
For example, let's add a printout *before* and *after* a function is called:

In [ ]:
def my_decorator(func): # takes a function as an argument
    def wrapper():
        print("Something is happening before the function is called.")
        func() # runs the function
        print("Something is happening after the function is called.")
    return wrapper # returns a function

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee) # redefines the original function

In [ ]:
say_whee()

Python allows you to use decorators in a simpler way with the `@` symbol:

In [ ]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

In [ ]:
say_whee()

### Useful libraries

You may want to have a look at the content of the following modules for further HO functions:

  - [operator](https://docs.python.org/3/library/operator.html)
  - [functools](https://docs.python.org/3/library/functools.html)
  - [itertools](https://docs.python.org/3/library/itertools.html)
  - [toolz](https://pypi.org/project/toolz/)
  - [funcy](https://pypi.org/project/funcy/)

# Classes and Objects

Object-oriented programming is a way to write programs so that properties and behaviours are tied to **objects**.

Classes are defined similarly to standard object-oriented languages, with similar functionalities.

The main Python documentation [page](https://docs.python.org/3.8/tutorial/classes.html) is worth reading.

Users can define new objects (classes) to extend the functionalities of primitive data structures like `int`, `float`, `list` and so on, to include other information. These variables, which belong to the class, are called **attributes**.

Classes are not only used to store data, but they also define functions called **methods**, which define the actions that an object created from the class can perform with its data.

It's important not to confuse two different concepts:

- A **class** defines a structure, but doesn't fill the content

- An **instance** is an object that is built from a class and contains real data.

In [ ]:
import math

# Class definition
class VectorND:
    '''This is a comment that is supposed to describe the purpose of the class'''
    
    # Definition of the class attributes, which are common for all instances of the same class
    x = []
    
    # Definition of the Constructor, a special method that is called every time a new object is created
    # The first argument of the constructor (and also for all other methods in the class) is the instance itself
    def __init__(self, components):
        self.x = components # a list is expected as input
    
    # Definition of the destructor, but it is often omitted
    #def __del__(self):
    #    print("Goodbye")
    
    # Definition of the methods
    
    # This method allows to get some info about the attribute of the class (x)
    def getDimension(self): # the first argument is always 'self' for the methods of a class
        return len(self.x)
    
    # This method allows to get individial elements of the 'x' attribute 
    def getX(self, n): # n is the component index
        return self.x[n]
    
    # This method allows to set individial elements of the 'x' attribute 
    def setX(self, n, xi): # n is the component index, and xi is the value
        if n < len(self.x):
            self.x[n] = xi
    
    # This method allows to perform non-trivial operations on the class attributes
    def getNorm(self):
        s2 = 0
        for i in range(len(self.x)):
            s2 += self.x[i]**2
        return math.sqrt(s2)

# End of the class definition

In [ ]:
# Create an instance of class 'VectorND'
a = VectorND([5, 0, 1])

print("Dimension:", a.getDimension()) # call method 'getDimension' of instance 'a'

a.setX(2, 0) # set the third component to zero

a.x[2] = 1 # another way (not recommended!) to modify the attributes of 'a'

print("Nornamlization:", a.getNorm())

Note that Python does not have private and public attributes. Outside the class definition, it is possible to access the class attributes even without defining the corresponding setter and getter methods.

### Class inheritance

Python allows to extend the functionality of a class by using *subclasses*, which are classes that **inherit** properties from the original class. In other words, the *child class* will inherit the properties of the *parent class*.

In [ ]:
class Vector3D(VectorND): # class 'Vector3D' inherits from class 'VectorND'
    
    # The constructor here is optional, and can be inherited from the parent class if omitted
    def __init__(self, components):
        if len(components) == 3:
            self.x = components # a list is expected as input
        else:
            print("Error: number of components is not 3")
    
    # New methods that only belongs to the child class
    def cross(self, y):
        return (self.x[1]*y.getX(2) - self.x[2]*y.getX(1), -self.x[0]*y.getX(2) + self.x[2]*y.getX(0), self.x[0]*y.getX(1) - self.x[1]*y.getX(0))

In [ ]:
b = Vector3D([1, 2, 3])
c = Vector3D([4, 5, 6])

# Child class methods are available to the child intance
d = b.cross(c)
print("Vector product:", d)

# Child instances also have the parent class' methods
print("Dimension:", b.getDimension())
print("Norm:", b.getNorm())

# But not the opposite
#a.cross(c) # AttributeError: 'Vector3D' object has no attribute 'vect'